In [1]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
import numpy as np

import sys
sys.path.append('../../')
import saedfsc

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
numCustomersTypes = 3
numSubsystems = 10
numObjectives = 11

# Create a list of supplier names
names = ['CustomerType' + str(i) for i in range(1, numCustomersTypes + 1)]

qty = np.random.randint(low=25, high=1000, size=numCustomersTypes)
meanDemand = np.random.randint(low=15, high=30, size=numCustomersTypes)
stdDevDemand = np.random.randint(low=1, high=5, size=numCustomersTypes)
priceFocus = np.random.random(size=numCustomersTypes)

rawPartworthWts = np.random.random(numSubsystems)
partworthWts = [np.array2string(rawPartworthWts/sum(rawPartworthWts), separator=',') for _ in range(numCustomersTypes)]
rawPerfWts = np.random.random(numObjectives)
perfWts = [np.array2string(rawPerfWts/sum(rawPerfWts), separator=',') for _ in range(numCustomersTypes)]

# Create the DataFrame
df = DataFrame({
    'Name': names,
    'Quantity': qty,
    'Mean Demand': meanDemand,
    'Std Dev Demand': stdDevDemand,
    'PriceFocus': priceFocus,
    'PartworthUtilityWeights': partworthWts,
    'PerformanceUtilityWeights': perfWts
})
df.to_csv('../resources/customers-{}.csv'.format(numCustomersTypes), index=False)
customers: DataFrame = read_csv('../resources/customers-{}.csv'.format(numCustomersTypes))
customers

,Name,Quantity,Mean Demand,Std Dev Demand,PriceFocus,PartworthUtilityWeights,PerformanceUtilityWeights
0,CustomerType1,369,28,3,0.687201,"[0.06897525,0.09315147,0.10288738,0.14439312,0...","[0.03042763,0.10658991,0.11520555,0.04257262,0..."
1,CustomerType2,764,17,3,0.921499,"[0.06897525,0.09315147,0.10288738,0.14439312,0...","[0.03042763,0.10658991,0.11520555,0.04257262,0..."
2,CustomerType3,927,25,4,0.327967,"[0.06897525,0.09315147,0.10288738,0.14439312,0...","[0.03042763,0.10658991,0.11520555,0.04257262,0..."


Print dictionary of customers and their weights.

In [3]:
name_weights_dict = customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
cWts

{'CustomerType1': array([0.03042763, 0.10658991, 0.11520555, 0.04257262, 0.09415735,
        0.06300855, 0.11802392, 0.10337506, 0.11301378, 0.12870588,
        0.08491975]),
 'CustomerType2': array([0.03042763, 0.10658991, 0.11520555, 0.04257262, 0.09415735,
        0.06300855, 0.11802392, 0.10337506, 0.11301378, 0.12870588,
        0.08491975]),
 'CustomerType3': array([0.03042763, 0.10658991, 0.11520555, 0.04257262, 0.09415735,
        0.06300855, 0.11802392, 0.10337506, 0.11301378, 0.12870588,
        0.08491975])}

In [4]:
cQty = dict(zip(customers['Name'], customers['Quantity'])) # customer quantities
cQty

{'CustomerType1': 369, 'CustomerType2': 764, 'CustomerType3': 927}